<a href="https://colab.research.google.com/github/Feterlike/colab-youtube-downloader/blob/main/UMD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 🎬 Universal Media Downloader (Drive or Local Device)
#@markdown ### 1. Paste the link (YouTube, X, Reddit, Instagram, etc.):
video_url = "" #@param {type:"string"}

#@markdown ---
#@markdown ### 2. Where do you want to save the file?
save_location = "Local Device" #@param ["Google Drive", "Local Device"]

#@markdown ---
#@markdown ### 3. Choose your desired quality (Max limit):
quality = "1080p" #@param ["best", "2160p", "1440p", "1080p", "720p", "480p", "360p", "worst", "audio_only"]

#@markdown ---
#@markdown ### 4. Click the Play Button (▶️) on the left to start.

# -------------------- DO NOT EDIT THE CODE BELOW --------------------

import os
import shutil
import glob
from google.colab import files, drive

# 1. Install/update yt-dlp quietly
print("⏳ Installing tools...")
!pip install -U yt-dlp --quiet

# 2. Configure Paths and Storage
base_folder = ""

if save_location == "Google Drive":
    try:
        drive.mount('/content/drive', force_remount=True)
        base_folder = "/content/drive/My Drive/UniversalDownloads"
        if not os.path.exists(base_folder):
            os.makedirs(base_folder)
    except Exception as e:
        print(f"❌ Error mounting drive: {e}")
        base_folder = "/content/temp_downloads" # Fallback
else:
    # Local Device logic: Save to a temp folder first
    base_folder = "/content/temp_downloads"
    if os.path.exists(base_folder):
        shutil.rmtree(base_folder) # Clean up previous runs
    os.makedirs(base_folder)

# 3. Validate input
if not video_url or not video_url.startswith("http"):
    print("❌ ERROR: Please paste a valid URL in the 'video_url' field.")
else:
    # --- Logic to handle AUDIO or VIDEO download ---

    # Common yt-dlp arguments
    # We use a user agent to prevent blocking from Twitter/Instagram
    ua = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"

    if quality == "audio_only":
        print(f"✅ Starting audio extraction...")
        print(f"   ▷ URL: {video_url}")
        cmd = f'yt-dlp -x --audio-format mp3 --no-warnings -q --add-header "User-Agent:{ua}" -P "{base_folder}" "{video_url}"'
        os.system(cmd)
        print(f"✅ Audio extraction complete!")

    else:
        # Video Logic
        if quality == "best":
            format_code = "bestvideo+bestaudio/best"
        elif quality == "worst":
            format_code = "worstvideo+worstaudio/worst"
        else:
            res = quality.replace('p', '')
            format_code = f"bestvideo[height<={res}]+bestaudio/best[height<={res}]/best"

        print(f"✅ Starting video download...")
        print(f"   ▷ URL: {video_url}")
        print(f"   ▷ Quality: {quality}")

        # --recode-video mp4 ensures compatibility for Reddit/Twitter files
        cmd = f'yt-dlp -f "{format_code}" --merge-output-format mp4 --recode-video mp4 --no-warnings -q --add-header "User-Agent:{ua}" -P "{base_folder}" "{video_url}"'
        os.system(cmd)
        print(f"✅ Video download complete!")

    # 4. Handle File Delivery
    if save_location == "Google Drive":
        print(f"   ▷ File saved to: {base_folder}")
    else:
        # Local Device: Find the file and trigger browser download
        print("   ▷ Preparing file for download to your device...")
        downloaded_files = glob.glob(f"{base_folder}/*")

        if downloaded_files:
            for file_path in downloaded_files:
                try:
                    files.download(file_path)
                    print(f"   ▷ Downloading: {os.path.basename(file_path)}")
                except Exception as e:
                    print(f"   ❌ Download failed: {e}")
        else:
            print("   ❌ Error: File not found. The download might have failed.")